In [1]:
import openpyxl
import pandas as pd
import os
import re
import json

overview_df = pd.read_excel("wi_overview_cleaned.xlsx")
overview_df['rapporteur'] = overview_df['rapporteur'].str.lower()
supporter_df = pd.read_excel("supporter_overview.xlsx")
supporter_df['rapporteur'] = supporter_df['rapporteur'].str.lower()
all_wi_df = pd.read_excel("Work_plan_3gpp_230110.xlsm")
# new_crawl = pd.read_excel("new_crawl.xlsx")
# obser_company_df = pd.read_excel("company_overview.xlsx")
# dyad_df = pd.read_excel("all_supporting_12_to_18.xlsx")
# date: 2023/2/19

In [2]:
# create dictionary need_replace from replace_name.ipynb

with open('replace_name.ipynb', 'r') as file:
    notebook_data = json.load(file)
    
cell = notebook_data['cells'][0]
code = cell['source']
exec(''.join(code))

## 1.  整理supporting member表格

In [8]:
all_wi_df[all_wi_df.Unique_ID == 920039]

print('沒有顯示在supporter overview當中的work item')
for index, row in supporter_df.iterrows():
    if row['unique_ID'] not in list(all_wi_df['Unique_ID']):
        print(row['unique_ID'])


In [1]:
target_rel = ['Rel-12', 'Rel-13', 'Rel-14', 'Rel-15', 'Rel-16','Rel-17', 'Rel-18']
dyad_df = pd.DataFrame()

none_data = []

for index, row in all_wi_df.iterrows():
    if row['Unique_ID'] != 0 and row['Unique_ID'] != 'nan' and row['Release'] in target_rel:
        if row['Unique_ID'] in list(supporter_df['unique_ID']):
            dyad_df = pd.concat([dyad_df, supporter_df[supporter_df.unique_ID == row['Unique_ID']][['unique_ID', 'release','ran','rapporteur','supporting member']]])
        
        elif row['Unique_ID'] not in list(supporter_df['unique_ID']):
            parent_wi = [wi for wi in list(all_wi_df[all_wi_df.Hyperlink == row['Hyperlink']].Unique_ID) if wi in list(supporter_df['unique_ID'])]
            
            if row['Hyperlink'] == '-' or len(row['Hyperlink']) > 10 or len(parent_wi) == 0:
                simple_df = pd.DataFrame({
                    'unique_ID':row['Unique_ID'],
                    'release': row['Release'],
                    'ran':overview_df[overview_df.Unique_ID == row['Unique_ID']]['TSG-RAN'],
                    'rapporteur':overview_df[overview_df.Unique_ID == row['Unique_ID']]['rapporteur']
                })
                dyad_df = pd.concat([dyad_df, simple_df]) 
            
            elif len(parent_wi) == 1:
                copy_data = supporter_df[supporter_df.unique_ID == parent_wi[0]][['rapporteur','supporting member']]
                copy_data.insert(0, 'ran', overview_df[overview_df.Unique_ID == row['Unique_ID']]['TSG-RAN'].item())
                copy_data.insert(0, 'release', row['Release'])
                copy_data.insert(0, 'unique_ID', row['Unique_ID'])
                dyad_df = pd.concat([dyad_df, copy_data])

            elif len(parent_wi) > 1:
#                 print(f"=========={row['Unique_ID']}============")
                for wi in parent_wi:
                    if overview_df[overview_df.Unique_ID == row['Unique_ID']]['rapporteur'].item() == overview_df[overview_df.Unique_ID == wi]['rapporteur'].item():
#                         print(f"{row['Unique_ID']} == {wi}")
                        copy_data = supporter_df[supporter_df.unique_ID == wi][['rapporteur','supporting member']]
                        display(copy_data)
                        copy_data.insert(0, 'ran', overview_df[overview_df.Unique_ID == row['Unique_ID']]['TSG-RAN'].item())
                        copy_data.insert(0, 'release', row['Release'])
                        copy_data.insert(0, 'unique_ID', row['Unique_ID'])
                        dyad_df = pd.concat([dyad_df, copy_data])                                           
                        break

In [20]:
dyad_df
dyad_df.to_excel('all_supporting_12_to_18.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [68]:
for index, row in all_wi_df.iterrows():
    if row['Release'] in target_rel and row['Unique_ID'] not in list(dyad_df['unique_ID']) and row['Unique_ID'] != 0 :
        print(row['Unique_ID'])

## 2. 統一WI表格中WIR與SUP企業名稱

In [5]:
overview_df.head(1)

,Unique_ID,Release,TSG-RAN,TSG-SA,TSG-CT,rapporteur,WI rapporteur name,WI rapporteur e-mail,is_hyperlink,is_download,Impacted TSs and TRs,Start,Created,last modif,Finish,duration,is_complete
0,959999,-,N,N,N,-,-,-,Y,N,NaN,2019-03-07,2022-03-13,NaT,2019-03-07,0,N


In [224]:
for key, value in need_replace.items():
    overview_df["rapporteur"] = overview_df["rapporteur"].str.replace(fr'^{key}$',need_replace[key], regex=True)

### 手動過濾合併

In [225]:
len(overview_df)

8143

In [165]:
# ========= step1: 將overview_df不在company清單的企業移除至 wi_overview_dirty.xlsx ========= 
# condition = overview_df['rapporteur'].isin(companies)
# dirty_df = overview_df[condition == False]
# print(len(dirty_df))
# overview_df.drop(dirty_df.index, inplace=True)
# dirty_df.to_excel('wi_overview_dirty.xlsx', encoding='utf_8_sig', index=False)

# ========= step2: 手動整理完之後，讀取cleaned_df再併入overview_df中 ========= 
# cleaned_df = pd.read_excel("wi_overview_dirty.xlsx")
# overview_df = pd.concat([overview_df, cleaned_df])
# print(len(cleaned_df))
# print(len(overview_df))

# ========= step3: 確認overview_df總筆術後，存成新檔 wi_overview_cleaned.xlsx ========= 
# overview_df.to_excel('wi_overview_cleaned.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


## 3. 統一supporting member表格中WIR與SUP企業名稱

In [11]:
len(supporter_df)

61099

In [28]:
for key, value in need_replace.items():
    supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(fr'^{key}$',need_replace[key], regex=True)


# 對指定欄位進行替換
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r'^[^a-z]*|[^a-z]*$', '', regex=True)
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r' ltd$', '', regex=True)
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r' limited$', '', regex=True)
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r' uk$', '', regex=True)  
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r' us$', '', regex=True)  
supporter_df["supporting member"] = supporter_df["supporting member"].str.replace(r' inc$', '', regex=True)  

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [7]:
supporter_df = supporter_df.drop_duplicates()
supporter_df.to_excel('supporter_overview.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


## 4. 合併WI表格到supporting member表格中

In [28]:
# ========= step1: 刪除重複資料、無效資料 ========= 

# 刪除Work item名稱、檔案名稱
supporter_df = supporter_df.drop(['title', 'file'], axis=1)

# 刪除重複值
supporter_df[supporter_df.duplicated() == True].to_excel('supporter_df_duplicated.xlsx', encoding='utf_8_sig', index=False)
supporter_df = supporter_df.drop_duplicates()
print(len(supporter_df))
supporter_df.head()

In [29]:
print(len(sup_release))
print(len(sup_ran))
print(len(sup_sa))
print(len(sup_ct))
print(len(sup_rapport))
print(len(sup_start))
print(len(sup_end))
print(len(sup_complete))
supporter_df = supporter_df.drop(['rapporteur'], axis=1)
# supporter_df.insert(3, 'complete', sup_complete)
# supporter_df.insert(3, 'end', sup_end)
# supporter_df.insert(3, 'start', sup_start)
supporter_df.insert(6, 'rapporteur', sup_rapport)

# supporter_df.insert(2, 'ct', sup_ct)
# supporter_df.insert(2, 'sa', sup_sa)
# supporter_df.insert(2, 'ran', sup_ran)
# supporter_df.insert(2, 'release', sup_release)
supporter_df.head(1)

In [303]:
supporter_df['duration'] = (supporter_df['end'] - supporter_df['start']).dt.days

In [23]:
supporter_df = supporter_df.drop_duplicates()
supporter_df.to_excel('supporter_overview.xlsx', encoding='utf_8_sig', index=False)

## 5. 整理company overview表格：所有企業在不同版本參與WI次數

### 針對所有release

In [3]:
target_rel = ['Rel-12', 'Rel-13', 'Rel-14', 'Rel-15', 'Rel-16','Rel-17', 'Rel-18']

all_rapporteurs = []
ran_rapporteurs = []

for index, row in overview_df.iterrows():
    if row['release'] in target_rel:
        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_rapporteurs += row["rapporteur"].split(', ')
            all_rapporteurs += row["rapporteur"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_rapporteurs.append(row['rapporteur'])
            all_rapporteurs.append(row['rapporteur'])
        
all_supporters = []
ran_supporters = []
all_supported_count = []
ran_supported_count = []

for index, row in supporter_df.iterrows():
    if row['release'] in target_rel:
        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_supporters += row["supporting member"].split(', ')
            all_supporters += row["supporting member"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_supporters.append(row['supporting member'])
            all_supporters.append(row['supporting member'])
            
        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_supported_count += row["rapporteur"].split(', ')
            all_supported_count += row["rapporteur"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_supported_count.append(row['rapporteur'])
            all_supported_count.append(row['rapporteur'])    
            

In [4]:
for key, value in need_replace.items():
    ran_rapporteurs = [value if name == key else name for name in ran_rapporteurs]
    all_rapporteurs = [value if name == key else name for name in all_rapporteurs]
    

In [5]:
ran_rapporteur = pd.Series(ran_rapporteurs).value_counts().reset_index()
ran_rapporteur = ran_rapporteur.rename(columns={'index': 'company', 0: '(RAN)rapporteur'})
all_rapporteur = pd.Series(all_rapporteurs).value_counts().reset_index()
all_rapporteur = all_rapporteur.rename(columns={'index': 'company', 0: 'rapporteur'})

ran_supported = pd.Series(ran_supported_count).value_counts().reset_index()
ran_supported = ran_supported.rename(columns={'index': 'company', 0: '(RAN)supported'})
all_supported = pd.Series(all_supported_count).value_counts().reset_index()
all_supported = all_supported.rename(columns={'index': 'company', 0: 'supported'})

ran_supporters = pd.Series(ran_supporters).value_counts().reset_index()
ran_supporters = ran_supporters.rename(columns={'index': 'company', 0: '(RAN)supporter'})
all_supporters = pd.Series(all_supporters).value_counts().reset_index()
all_supporters = all_supporters.rename(columns={'index': 'company', 0: 'supporter'})

all_counts = pd.merge(all_rapporteur,all_supported,on='company',how='outer')
all_counts = pd.merge(all_counts,all_supporters,on='company',how='outer')
ran_counts = pd.merge(ran_rapporteur,ran_supported,on='company',how='outer')
ran_counts = pd.merge(ran_counts,ran_supporters,on='company',how='outer')
print(ran_counts.head(2))
print(all_counts.head(2))

    company  (RAN)rapporteur  (RAN)supported  (RAN)supporter
0    huawei            339.0          3917.0            1439
1  ericsson            298.0          4293.0            1404
    company  rapporteur  supported  supporter
0    huawei       737.0     8456.0     3077.0
1  ericsson       658.0     7553.0     3134.0


### 針對特定release

In [6]:
company_count_by_release = {}

for rel in range(12, 19):
    overview_df_by_rel = overview_df[overview_df.release == f'Rel-{rel}']
    supporter_df_by_rel = supporter_df[supporter_df.release == f'Rel-{rel}']
    
    all_rapporteurs = []
    ran_rapporteurs = []
   
    for index, row in overview_df_by_rel.iterrows():

        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_rapporteurs += row["rapporteur"].split(', ')
            all_rapporteurs += row["rapporteur"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_rapporteurs.append(row['rapporteur'])
            all_rapporteurs.append(row['rapporteur'])
    
    
    all_supporters = []
    ran_supporters = []
    all_supported = []
    ran_supported = []
    
    for index, row in supporter_df_by_rel.iterrows():

        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_supporters += row["supporting member"].split(', ')
            all_supporters += row["supporting member"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_supporters.append(row['supporting member'])
            all_supporters.append(row['supporting member'])
                    
        if ',' in str(row["rapporteur"]):
            if row['ran'] == 'Y':
                ran_supported += row["rapporteur"].split(', ')
            all_supported += row["rapporteur"].split(', ')
        else:
            if row['ran'] == 'Y':
                ran_supported.append(row['rapporteur'])
            all_supported.append(row['rapporteur'])    
            
    company_count_by_release[f'rapporteur-{rel}'] = all_rapporteurs
    company_count_by_release[f'(RAN)rapporteur-{rel}'] = ran_rapporteurs
    company_count_by_release[f'supported-{rel}'] = all_supported
    company_count_by_release[f'(RAN)supported-{rel}'] = ran_supported
    company_count_by_release[f'supporter-{rel}'] = all_supporters
    company_count_by_release[f'(RAN)supporter-{rel}'] = ran_supporters

### 拆分成TSG和RAN兩個版本
1. TSG: 企業在不同release，參與所有會議次數
2. RAN: 企業在不同release，參與RAN會議次數

In [8]:
for title in ['rapporteur-','supported-','supporter-']:
    for rel in range(12, 19):
#         print(f'{title}{rel}')
        count = pd.Series(company_count_by_release[f'{title}{rel}']).value_counts().reset_index()
        count = count.rename(columns={'index': 'company', 0: f'{title}{rel}'})
        all_counts = pd.merge(all_counts,count,on='company',how='outer')

for title in ['(RAN)rapporteur-','(RAN)supported-','(RAN)supporter-']:
    for rel in range(12, 19):
#         print(f'{title}{rel}')
        count = pd.Series(company_count_by_release[f'{title}{rel}']).value_counts().reset_index()
        count = count.rename(columns={'index': 'company', 0: f'{title}{rel}'})
        ran_counts = pd.merge(ran_counts,count,on='company',how='outer')
        
display(ran_counts.head(2))
display(all_counts.head(2))

,company,(RAN)rapporteur,(RAN)supported,(RAN)supporter,(RAN)rapporteur-12,(RAN)rapporteur-13,(RAN)rapporteur-14,(RAN)rapporteur-15,(RAN)rapporteur-16,(RAN)rapporteur-17,...,(RAN)supported-16,(RAN)supported-17,(RAN)supported-18,(RAN)supporter-12,(RAN)supporter-13,(RAN)supporter-14,(RAN)supporter-15,(RAN)supporter-16,(RAN)supporter-17,(RAN)supporter-18
0,huawei,339.0,3917.0,1439,43.0,94.0,27.0,41.0,56.0,46.0,...,706.0,518.0,518.0,214.0,276.0,116.0,119.0,262.0,254.0,198.0
1,ericsson,298.0,4293.0,1404,44.0,53.0,45.0,40.0,53.0,33.0,...,653.0,581.0,497.0,304.0,286.0,127.0,98.0,239.0,202.0,148.0


,company,rapporteur,supported,supporter,rapporteur-12,rapporteur-13,rapporteur-14,rapporteur-15,rapporteur-16,rapporteur-17,...,supported-16,supported-17,supported-18,supporter-12,supporter-13,supporter-14,supporter-15,supporter-16,supporter-17,supporter-18
0,huawei,737.0,8456.0,3077.0,106.0,137.0,66.0,100.0,113.0,127.0,...,1424.0,1471.0,1086.0,504.0,485.0,292.0,270.0,497.0,604.0,425.0
1,ericsson,658.0,7553.0,3134.0,142.0,107.0,79.0,69.0,84.0,99.0,...,933.0,1343.0,963.0,619.0,517.0,330.0,272.0,488.0,549.0,359.0


In [9]:
to_drop = all_counts[all_counts['company'] == '-'].index
all_counts = all_counts.drop(to_drop)

all_counts.to_excel('all_tsg_company_overview.xlsx', encoding='utf_8_sig', index=False)

to_drop = ran_counts[ran_counts['company'] == '-'].index
ran_counts = ran_counts.drop(to_drop)

ran_counts.to_excel('ran_company_overview.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
